## Point 2 :  Modify your event driven workflow: can you rewrite your Lambda functions so that the workflow can process multiple image inputs in parallel? Can the Step Function "fan out" to accomodate this new workflow?

#### Response
- No lambda function code has been replaced. 
- The map feature of the step function service has been used instead.

---
### Import

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [85]:
!pip install jsonlines

In [86]:
from uuid import uuid4
import random, json, jsonlines
from time import strftime
from datetime import datetime as dt

In [76]:
import sagemaker
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()
region = sess.boto_region_name

import boto3, os

client_s3 = boto3.Session().resource('s3')



In [4]:
sess.default_bucket(), region, role

('sagemaker-us-east-1-416312206633',
 'us-east-1',
 'arn:aws:iam::416312206633:role/service-role/AmazonSageMaker-ExecutionRole-20211216T204468')

In [5]:
working_bucket = 'batch-classification-scones-project'

In [432]:
#!aws s3 rm s3://batch-classification-scones-project --recursive

In [ ]:
#!aws s3 rm s3://batch-classification-scones-project/train_batch_classification --recursive
#!aws s3 rm s3://batch-classification-scones-project/test_batch_classification --recursive
    

In [433]:
!rm -rf ./train_batch_classification
!rm -rf ./test_batch_classification

In [6]:
import pickle

with open("./cifar-100-python/meta", "rb") as f:
    dataset_meta = pickle.load(f, encoding='bytes')

with open("./cifar-100-python/test", "rb") as f:
    dataset_test = pickle.load(f, encoding='bytes')

with open("./cifar-100-python/train", "rb") as f:
    dataset_train = pickle.load(f, encoding='bytes')

In [7]:
pd.DataFrame(dataset_train[b'fine_labels'])

,0
0,19
1,29
2,0
3,11
4,1
...,...
49995,80
49996,7
49997,3
49998,7


In [8]:
pd_fine_labels = pd.DataFrame(dataset_train[b'fine_labels'])
pd.crosstab(index=pd_fine_labels[0], columns="count")

col_0,count
0,
0,500
1,500
2,500
3,500
4,500
...,...
95,500
96,500
97,500


In [9]:
pd_fine_labels.value_counts()

99    500
36    500
26    500
27    500
28    500
     ... 
69    500
70    500
71    500
72    500
0     500
Length: 100, dtype: int64

In [10]:
#print(dataset_meta[b'fine_label_names'][dataset_train[b'fine_labels'][n]])
pd.DataFrame(dataset_train[b'fine_labels'])

,0
0,19
1,29
2,0
3,11
4,1
...,...
49995,80
49996,7
49997,3
49998,7


In [11]:
dataset_meta[b'fine_label_names']

[b'apple',
 b'aquarium_fish',
 b'baby',
 b'bear',
 b'beaver',
 b'bed',
 b'bee',
 b'beetle',
 b'bicycle',
 b'bottle',
 b'bowl',
 b'boy',
 b'bridge',
 b'bus',
 b'butterfly',
 b'camel',
 b'can',
 b'castle',
 b'caterpillar',
 b'cattle',
 b'chair',
 b'chimpanzee',
 b'clock',
 b'cloud',
 b'cockroach',
 b'couch',
 b'crab',
 b'crocodile',
 b'cup',
 b'dinosaur',
 b'dolphin',
 b'elephant',
 b'flatfish',
 b'forest',
 b'fox',
 b'girl',
 b'hamster',
 b'house',
 b'kangaroo',
 b'keyboard',
 b'lamp',
 b'lawn_mower',
 b'leopard',
 b'lion',
 b'lizard',
 b'lobster',
 b'man',
 b'maple_tree',
 b'motorcycle',
 b'mountain',
 b'mouse',
 b'mushroom',
 b'oak_tree',
 b'orange',
 b'orchid',
 b'otter',
 b'palm_tree',
 b'pear',
 b'pickup_truck',
 b'pine_tree',
 b'plain',
 b'plate',
 b'poppy',
 b'porcupine',
 b'possum',
 b'rabbit',
 b'raccoon',
 b'ray',
 b'road',
 b'rocket',
 b'rose',
 b'sea',
 b'seal',
 b'shark',
 b'shrew',
 b'skunk',
 b'skyscraper',
 b'snail',
 b'snake',
 b'spider',
 b'squirrel',
 b'streetcar',
 b

In [12]:
#types_vehicles = [b'bus', b'bicycle', b'pickup_truck', b'rocket', b'motorcycle', b'streetcar', b'tank', b'tractor', b'train']
types_vehicles = [b'bicycle', b'motorcycle',  b'streetcar', b'rocket', b'tractor']

In [13]:
# liste 1 : dataset_meta[b'fine_label_names']
# liste 2 : type_vehicles

# Indices list of matching element from other list
# Using list comprehension + set() + enumerate()
temp = set(types_vehicles)
numbers_type_vehicle = [i for i, val in enumerate(dataset_meta[b'fine_label_names']) if val in temp]
numbers_type_vehicle

[8, 48, 69, 81, 89]

In [14]:
#Construct the dataframe
df_train = pd.DataFrame({
    "filenames": dataset_train[b'filenames'],
    "labels": dataset_train[b'fine_labels'],
    "row": range(len(dataset_train[b'filenames']))
})

# Drop all rows from df_train where label is not in types_vehicles
df_train = df_train[df_train['labels'].isin(numbers_type_vehicle)]

# Decode df_train.filenames so they are regular strings
df_train["filenames"] = df_train["filenames"].apply(
    lambda x: x.decode("utf-8")
)


df_test = pd.DataFrame({
    "filenames": dataset_test[b'filenames'],
    "labels": dataset_test[b'fine_labels'],
    "row": range(len(dataset_test[b'filenames']))
})

# Drop all rows from df_test where label is not in types_vehicles
df_test = df_test[df_test['labels'].isin(numbers_type_vehicle)]

# Decode df_test.filenames so they are regular strings
df_test["filenames"] = df_test["filenames"].apply(
    lambda x: x.decode("utf-8")
)

In [15]:
df_train["filenames"].shape

(2500,)

In [16]:
df_test["filenames"].shape

(500,)

In [17]:
df_train.head(15)

,filenames,labels,row
16,bike_s_000682.png,8,16
30,bike_s_000127.png,8,30
36,streetcar_s_000663.png,81,36
48,tractor_s_001723.png,89,48
73,tractor_s_000104.png,89,73
76,bulldozer_s_000433.png,89,76
93,minuteman_s_000073.png,69,93
107,icbm_s_000392.png,69,107
130,cycle_s_002598.png,8,130
152,minibike_s_000824.png,48,152


In [18]:
df_test.head(15)

,filenames,labels,row
12,streetcar_s_000382.png,81,12
13,sounding_rocket_s_000010.png,69,13
27,safety_bike_s_000390.png,8,27
28,bike_s_000658.png,8,28
59,exocet_s_000376.png,69,59
85,dozer_s_001228.png,89,85
105,research_rocket_s_001220.png,69,105
116,velocipede_s_001744.png,8,116
161,bike_s_000643.png,8,161
219,streetcar_s_000124.png,81,219


In [19]:
!mkdir -p ./train_batch_classification/images
!mkdir -p ./test_batch_classification/images

In [22]:
def save_images(idx_image, channel, dataset):
    #Grab the image data in row-major form
    img = dataset[b'data'][idx_image]
    
    # Consolidated stacking/reshaping from earlier
    target = np.dstack((
            img[:1024].reshape(32, 32),
            img[1024:2048].reshape(32, 32),
            img[2048:].reshape(32, 32)
    ))
    
    # Save the image
    plt.imsave(f"./{channel}/images/{dataset[b'filenames'][idx_image].decode('utf-8')}", target)
    
    # Return any signal data you want for debugging
    return

for idx in df_train.index.values:
    save_images(idx, 'train_batch_classification', dataset_train)
    
for idx in df_test.index.values:
    save_images(idx, 'test_batch_classification', dataset_test)

In [24]:
import os

os.environ["WORKING_BUCKET"] = working_bucket
!aws s3 sync ./train_batch_classification s3://${WORKING_BUCKET}/train_batch_classification/
!aws s3 sync ./test_batch_classification s3://${WORKING_BUCKET}/test_batch_classification/

In [26]:
numbers_type_vehicle

[8, 48, 69, 81, 89]

In [27]:
types_vehicles

[b'bicycle', b'motorcycle', b'streetcar', b'rocket', b'tractor']

In [28]:
types_vehicles = list(zip(numbers_type_vehicle, types_vehicles))
[type for type in types_vehicles]

[(8, b'bicycle'),
 (48, b'motorcycle'),
 (69, b'streetcar'),
 (81, b'rocket'),
 (89, b'tractor')]

In [29]:
types_vehicles[-1]

(89, b'tractor')

In [30]:
numbers_type_vehicle

[8, 48, 69, 81, 89]

In [31]:
df_test["labels"]

12      81
13      69
27       8
28       8
59      69
        ..
9922    89
9928     8
9939     8
9946    69
9969    89
Name: labels, Length: 500, dtype: int64

--- 
### Model Training

For Image Classification, Sagemaker [also expects metadata](https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification.html) e.g. in the form of TSV files with labels and filepaths. We can generate these using our Pandas DataFrames from earlier:

In [32]:
def return_index_vehicle(number_vehicle):
    index_vehicle = 0
    while number_vehicle != numbers_type_vehicle[index_vehicle]:
        index_vehicle += 1
    return index_vehicle

In [33]:
#return_index_vehicle(85)

In [35]:
def to_metadata_file(df, prefix):
    df["s3_path"] = df["filenames"]
    df["labels"] = df["labels"].apply(lambda x: return_index_vehicle(x))
    return df[["row", "labels", "s3_path"]].to_csv(
        f"{prefix}.lst", sep="\t", index=False, header=False
    )
    
to_metadata_file(df_train.copy(), "train_batch_classification")
to_metadata_file(df_test.copy(), "test_batch_classification")

In [36]:
# Upload .lst files
client_s3.Bucket(working_bucket).Object('train_batch_classification.lst').upload_file('./train_batch_classification.lst')
client_s3.Bucket(working_bucket).Object('test_batch_classification.lst').upload_file('./test_batch_classification.lst')

In [37]:
from sagemaker import image_uris

# Use the image_uris function to retrieve the latest 'image-classification' image 
algo_image = image_uris.retrieve(framework='image-classification', region=region)
s3_output_location = f"s3://{working_bucket}/models/image_model"

In [38]:
s3_output_location

's3://batch-classification-scones-project/models/image_model'

In [39]:
# The local path where the model will save its checkpoints in the training container
checkpoint_local_path="/opt/ml/checkpoints" 

# The S3 URI to store the checkpoints
checkpoint_s3_bucket=f"s3://{working_bucket}/checkpoints"

#### Choice of parameters

In [41]:
#parameters for spot instance
use_spot_instances = True # Not simple to have spot capacity. Have to try many times. Perhaps I would try in another region, but all would have to be moved (s3...) 
max_run = 3600
max_wait = 7200 if use_spot_instances else None

In [42]:
#supported instances for image classification prebuilt sagemaker algorithms
# 'ml.p2.xlarge', 'ml.p2.8xlarge', 'ml.p2.16xlarge', 'ml.p3.2xlarge', 'ml.p3.8xlargeand', 'ml.p3.16xlarge'
instance_training_type = 'ml.p3.2xlarge'

In [ ]:
model_uuid = uuid4()

In [43]:
#supp

img_classifier_model=sagemaker.estimator.Estimator(
    ## TODO: define your estimator options  
    algo_image,
    role,
    instance_count= 1,
    #instance_type= 'ml.p2.xlarge',
    instance_type= instance_training_type,
    use_spot_instances= use_spot_instances,
    max_run= max_run,
    max_wait = max_wait,    
    output_path= s3_output_location,
    #name=
    #checkpoint_s3_uri=checkpoint_s3_bucket,    
    #checkpoint_local_path=checkpoint_local_path   
)

In [44]:
len(numbers_type_vehicle)

5

In [466]:
#img_classifier_model.set_hyperparameters(
#    image_shape= '3,32,32', # TODO: Fill in
#    num_classes=2, # TODO: Fill in
#    num_training_samples=1000, # TODO: fill in
#    learning_rate=0.05
#)

In [45]:
num_training_samples=2500

In [46]:
# hyperparameters for training
img_classifier_model.set_hyperparameters(
    image_shape= '3,32,32', # TODO: Fill in
    num_classes= len(numbers_type_vehicle), # TODO: Fill in
    num_training_samples= num_training_samples, # TODO: fill in
    learning_rate=0.01, 
    mini_batch_size= 64,
    epochs=50
)

#{'num_classes': '3', 'num_training_samples': '1500', 'image_shape': '3,32,32', 'learning_rate': '0.01'}

In [47]:
train_input = f"s3://{working_bucket}/train_batch_classification/images/"
test_input  = f"s3://{working_bucket}/test_batch_classification/images/"
train_lst  = f"s3://{working_bucket}/train_batch_classification.lst"
test_lst  = f"s3://{working_bucket}/test_batch_classification.lst"

In [48]:
print(train_input)
print(test_input)
print(train_lst)
print(test_lst)

s3://batch-classification-scones-project/train_batch_classification/images/
s3://batch-classification-scones-project/test_batch_classification/images/
s3://batch-classification-scones-project/train_batch_classification.lst
s3://batch-classification-scones-project/test_batch_classification.lst


In [49]:
from sagemaker.debugger import Rule, rule_configs
from sagemaker.session import TrainingInput
model_inputs = {
        "train": sagemaker.inputs.TrainingInput(
            s3_data=train_input,
            content_type="application/x-image"
        ),
        "validation": sagemaker.inputs.TrainingInput(
            s3_data=test_input,
            content_type="application/x-image"
        ),
        "train_lst": sagemaker.inputs.TrainingInput(
            s3_data=train_lst,
            content_type="application/x-image"
        ),
        "validation_lst": sagemaker.inputs.TrainingInput(
            s3_data=test_lst,
            content_type="application/x-image"
        )
}

In [50]:
img_classifier_model.fit(model_inputs)

2022-01-30 18:42:17 Starting - Starting the training job...
2022-01-30 18:42:47 Starting - Launching requested ML instancesProfilerReport-1643568137: InProgress
......
2022-01-30 18:43:47 Starting - Preparing the instances for training........
2022-01-30 18:45:07 Downloading - Downloading input data...
2022-01-30 18:45:47 Training - Downloading the training image......
2022-01-30 18:46:48 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
[01/30/2022 18:46:40 INFO 140223575467840] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/image_classification/default-input.json: {'use_pretrained_model': 0, 'num_layers': 152, 'epochs': 30, 'learning_rate': 0.1, 'lr_scheduler_factor': 0.1, 'optimizer': 'sgd', 'momentum': 0, 'weight_decay': 0.0001, 'beta_1': 0.9, 'beta_2': 0.999, 'eps': 1e-08, 'gamma': 0.9, 'mini_batch_size': 32, 'image_shape': '3,224,224', 'precision_dtype': 'float32'}
[01/30/2022 18:46:40 I

--- 

### Deployment



In [53]:
f"s3://{working_bucket}/data_capture"

's3://batch-classification-scones-project/data_capture'

In [55]:
from sagemaker.model_monitor import DataCaptureConfig

data_capture_config = DataCaptureConfig(
    enable_capture= True,
    sampling_percentage= 100,
    destination_s3_uri= f"s3://{working_bucket}/data_capture",
)

In [65]:
f'batch_classification-TIME-{dt.now().strftime("%Y-%m-%d-%H-%M-%S-%f")}-ID-{str(uuid4())}'

'batch_classification-TIME-2022-01-30-19-34-46-911066-ID-5b7491b7-bbc2-4bcd-b8d0-4481acb9fb04'

In [68]:
deployment = img_classifier_model.deploy(
    data_capture_config=data_capture_config,
    initial_instance_count= 1, 
    instance_type= 'ml.t2.medium', 
    endpoint_name= f'batch-classification-TIME-{dt.now().strftime("%Y-%m-%d-%H-%M-%S-%f")}'
    )

endpoint = deployment.endpoint_name
print(endpoint)

-----------------------!batch-classification-TIME-2022-01-30-19-36-14-465184


--- 
#### Predictions

In [88]:
#endpoint_name="batch-classification-TIME-2022-01-30-19-36-14-465184"

In [89]:
from sagemaker.predictor import Predictor

predictor = Predictor(endpoint)

In [90]:
from sagemaker.serializers import IdentitySerializer

predictor.serializer = IdentitySerializer("image/png")
with open("./test/images/bicycle_s_001789.png", "rb") as f:
    payload = f.read()

inference = predictor.predict(payload)

In [91]:
print(inference)

b'[0.9782111048698425, 0.013572802767157555, 0.006240911316126585, 0.00013658084208145738, 0.0018385974690318108]'


In [92]:
types_vehicles

[(8, b'bicycle'),
 (48, b'motorcycle'),
 (69, b'streetcar'),
 (81, b'rocket'),
 (89, b'tractor')]

--- 
#### Data capture

In [93]:
local_path_of_data_capture = './batch_classification/captured_data'

In [100]:
data_path

's3://batch-classification-scones-project/data_capture/'

In [101]:
from sagemaker.s3 import S3Downloader

# In S3 your data will be saved to a datetime-aware path
# Find a path related to a datetime you're interested in
data_path = f"s3://{working_bucket}/data_capture/" 

S3Downloader.download(data_path, local_path_of_data_capture)



In [102]:
file_handles = os.listdir(local_path_of_data_capture)
print(file_handles)

['batch-classification-TIME-2022-01-30-19-36-14-465184']


In [120]:
json_data = []
# List the file names we downloaded
for root, dirs, files in os.walk(local_path_of_data_capture, topdown=False):
    for name in files:
        with jsonlines.open(os.path.join(root, name)) as f:
            for line in f:
                json_data.append(line)
         #       print(line)
        #print("###############" + os.path.join(root, name))

In [ ]:
'''
json_data = []
# List the file names we downloaded
for root, dirs, files in os.walk(local_path_of_data_capture, topdown=False):
    for name in files:
        with jsonlines.open(os.path.join(root, name)) as f:
            json_data.append(f.read())
        
        print(os.path.join(root, name))
'''        

In [150]:
len(json_data)

290

In [146]:
def simple_getter(obj):
    inferences = obj["captureData"]["endpointOutput"]["data"]
    timestamp = obj["eventMetadata"]["inferenceTime"]
    return json.loads(inferences), timestamp

range(len(json_data))
list_max_inferences = [max(simple_getter(json_data[idx])[0]) for idx in range(len(json_data))]
probabilities= [simple_getter(json_data[idx])[0] for idx in range(len(json_data))]
df_probabilities_result = pd.DataFrame(probabilities)
df_probabilities_result.describe()

,0,1,2,3,4
count,2.900000e+02,2.900000e+02,2.900000e+02,2.900000e+02,2.900000e+02
mean,1.897513e-01,2.105317e-01,1.736046e-01,2.189568e-01,2.071557e-01
std,3.508609e-01,3.699147e-01,3.506887e-01,3.621098e-01,3.544176e-01
min,3.527180e-10,4.916572e-11,3.973015e-12,5.445946e-11,1.010907e-10
25%,8.729072e-06,1.207267e-05,5.960154e-07,1.024017e-05,3.855163e-06
50%,7.856085e-04,1.818875e-03,5.460895e-05,1.274047e-03,3.262654e-03
75%,1.112920e-01,2.055986e-01,4.117918e-02,2.835635e-01,1.708589e-01
max,1.000000e+00,1.000000e+00,1.000000e+00,9.999930e-01,9.999484e-01


In [154]:
#list_max_inferences[8]
probabilities[8]

[0.30821916460990906,
 0.0011776452884078026,
 0.003933131229132414,
 0.6632261872291565,
 0.02344389632344246]

In [155]:
df_probabilities_result.head(20).round(2)

,0,1,2,3,4
0,0.98,0.01,0.01,0.00,0.00
1,0.00,1.00,0.00,0.00,0.00
2,0.02,0.87,0.00,0.00,0.11
3,0.00,1.00,0.00,0.00,0.00
4,0.02,0.87,0.00,0.00,0.11
5,0.98,0.01,0.01,0.00,0.00
6,0.00,1.00,0.00,0.00,0.00
7,1.00,0.00,0.00,0.00,0.00
8,0.31,0.00,0.00,0.66,0.02
9,0.02,0.87,0.00,0.00,0.11
